In [1]:
import pandas as pd
import numpy as np

In [2]:
col_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

df_msl = pd.read_csv('https://raw.githubusercontent.com/IDno64261/timeseries/main/Busan_MSL_data_1961_2022_meanx.csv', skiprows=1, names=col_names)
df_msl = df_msl.apply(pd.to_numeric, errors='coerce')
df_msl.dropna(inplace=True)
df_msl.replace({-99.90 : np.NaN}, inplace=True)
df_msl

,January,February,March,April,May,June,July,August,September,October,November,December
1961,61.8,63.9,63.4,67.1,73.4,79.5,76.5,79.0,74.8,71.2,63.8,58.9
1962,58.5,49.6,49.0,51.5,57.7,68.8,73.0,73.0,74.7,68.2,65.9,60.8
1963,57.3,51.4,56.7,54.3,60.0,71.6,69.8,76.2,73.3,62.6,59.7,55.6
1964,58.4,52.3,52.6,54.0,67.2,70.2,67.3,78.0,81.1,68.6,64.0,60.9
1965,58.9,54.4,58.7,50.8,59.5,69.2,69.7,67.6,70.8,61.5,64.6,55.1
...,...,...,...,...,...,...,...,...,...,...,...,...
2018,63.2,58.5,59.5,64.3,75.2,76.9,81.6,89.8,83.7,77.2,69.3,71.9
2019,71.8,65.8,70.7,71.1,75.0,80.9,84.9,88.7,84.7,85.1,71.5,67.5
2020,73.6,70.4,67.9,68.7,77.0,86.8,85.4,81.8,92.0,83.9,68.8,68.8
2021,70.0,69.8,67.4,69.3,77.3,82.1,88.1,96.9,90.1,83.8,70.3,65.2


In [6]:
stacked_series = df_msl.stack()

df_msl_stacked = stacked_series.to_frame()
df_msl_stacked.reset_index(inplace=True)
df_msl_stacked.columns = ['Year', 'Month', 'MSL']

df_msl_stacked['Date'] = pd.to_datetime(df_msl_stacked['Year'].astype(str) + '-' + df_msl_stacked['Month'].astype(str))
df_msl_stacked['Date'] = df_msl_stacked['Date'].dt.strftime('%Y-%m')
df_msl_stacked.set_index('Date', inplace=True)
df_msl_stacked.drop(['Year', 'Month'], axis=1, inplace=True)
df_msl_stacked

<ipython-input-6-bc932904d7d3>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_msl_stacked['Date'] = pd.to_datetime(df_msl_stacked['Year'].astype(str) + '-' + df_msl_stacked['Month'].astype(str))


,MSL
Date,
1961-01,61.8
1961-02,63.9
1961-03,63.4
1961-04,67.1
1961-05,73.4
...,...
2022-08,83.1
2022-09,91.8
2022-10,76.9


In [8]:
import plotly.express as px

fig = px.line(df_msl_stacked, facet_col_spacing = 0.013)
fig.show()

In [9]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss

In [10]:
# ADF 테스트 실행 함수
def adf_test(timeseries):
    print('Results of Augmented Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(), autolag='AIC')  # NaN 값 제거 후 테스트
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput[f'Critical Value ({key})'] = value
    print(dfoutput)

# ADF 테스트 실행
adf_test(df_msl_stacked['MSL'])

Results of Augmented Dickey-Fuller Test:
Test Statistic                  -1.406233
p-value                          0.579284
#Lags Used                      19.000000
Number of Observations Used    712.000000
Critical Value (1%)             -3.439568
Critical Value (5%)             -2.865608
Critical Value (10%)            -2.568936
dtype: float64


In [11]:
# KPSS 테스트 실행 함수
def kpss_test(timeseries):
    print('\nResults of KPSS Test:')
    kpsstest = kpss(timeseries.dropna(), regression='c')  # NaN 값 제거 후 테스트
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic', 'p-value', 'Lags Used'])
    for key, value in kpsstest[3].items():
        kpss_output[f'Critical Value ({key})'] = value
    print(kpss_output)

    # KPSS 테스트 실행
kpss_test(df_msl_stacked['MSL'])


Results of KPSS Test:
Test Statistic            5.078554
p-value                   0.010000
Lags Used                11.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


<ipython-input-11-bf8e1d237c5e>:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


